In [1]:
import sys
sys.executable

'c:\\Users\\david\\AppData\\Local\\Programs\\Python\\Python311\\python.exe'

In [2]:
from huggingface_hub import login
login(token="") # set your huggingface token

In [3]:
from torch import cuda
model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

cuda:0


In [4]:
MODEL = 'Llama2-7b'
MODEL_FullName = 'meta-llama/Llama-2-7b-chat-hf'

In [5]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [6]:
from torch import bfloat16
import transformers
# Llama 3 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 3 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # This will ensure the model is loaded to GPU
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096

In [7]:
# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

Device set to use cuda:0


In [8]:
import pandas as pd
import ast
df= pd.read_csv('pull_requests_filtered_raw.csv')
df['comments'] = df['comments'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['review_comments'] = df['comments'].apply(lambda comments: [item for item in comments if item['type'] != 'issue'] if type(comments) is not float else comments)
df = df[df['review_comments'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
df['issue_comments'] = df['comments'].apply(lambda comments: [item for item in comments if item['type'] != 'review'] if type(comments) is not float else comments)
df = df[df['issue_comments'].apply(lambda x: isinstance(x, list) and len(x) > 0)]


In [9]:
import re
def clean_text(text):

    pattern = r"```.*?```|http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    return re.sub(pattern, lambda m: "" if m.group(0).startswith("```") else "", text, flags=re.DOTALL)

def extract_text(comment_thread):
    
    conversation = []
    for comment in comment_thread:
        main_comment = clean_text(comment.get('comment', {}).get('body', ''))
        
        # Extract replies' bodies
        replies = comment.get('replies', [])
        reply_bodies = [reply.get('body', '') for reply in replies]

        if main_comment:
            conversation.append(main_comment)

        for r in reply_bodies:
            cleaned_reply = clean_text(r)  # Apply regex cleaning
            if cleaned_reply.strip():  # Ensure we don't add empty strings
                conversation.append(cleaned_reply)
            
    return conversation
# Apply the function to extract text from the comments column
df['processed_comments_review'] = df['review_comments'].apply(extract_text)
print(df['processed_comments_review'].iloc[1])
df['processed_comments_issue'] = df['issue_comments'].apply(extract_text)
print(df['processed_comments_issue'].iloc[1])

# extract_text(df['review_comments'][0])


["This are glob pattern not regexes, so the {6} doesn't work afaik. The [0-9] appears to be fine though. ", 'May be parametrize some hostnames to make sure all regex expressions are tested.', "I don't understand? I mean we are not evaluating the glob patterns here anyways, so we are just passing through the strings here. ", "It's not a bad idea though (added it)"]
['Do we need a docs PR here?', '> Do we need a docs PR here?\r\n\r\nI added a note to docs.']


In [10]:
df = df[['PR Number', 'processed_comments_review', 'processed_comments_issue']].reset_index(drop=True)
df.head()


,PR Number,processed_comments_review,processed_comments_issue
0,129755,"[Tests are missing., Waiting on an intents bum...",[Why is this added to the November release mil...
1,129675,"[This are glob pattern not regexes, so the {6}...","[Do we need a docs PR here?, > Do we need a do..."
2,129299,[Please reduce to one platform #home-assistant...,[I made the single PR for the whole lg_thinq i...
3,129232,[I think a few of these are bugfixes that shou...,[There are quite a few tests which does a reau...
4,129088,[This can now be moved to a constant at the to...,[@epenet What's the advantage of `suggested_va...


In [17]:
df['processed_comments_review'].tolist()

['Tests are missing. Waiting on an intents bump before the test can be added. Tests added in ',
 "This are glob pattern not regexes, so the {6} doesn't work afaik. The [0-9] appears to be fine though.  May be parametrize some hostnames to make sure all regex expressions are tested. I don't understand? I mean we are not evaluating the glob patterns here anyways, so we are just passing through the strings here.  It's not a bad idea though (added it)",
 "Please reduce to one platform #home-assistant-core Everything has been merged to an intermediary branch and already has been reviewed thoroughly :)\r\n\r\n Oh! Forget I said anything then :) Thanks for staying sharp ❤️  Joost! Is this going in for this beta? I've a washer to test, in case! :) That is the plan yes :) > Joost! Is this going in for this beta? I've a washer to test, in case! :)\r\n\r\nThank u for the washer.\r\nI hope it works :)",
 'I think a few of these are bugfixes that should probably be moved to separate prs I have made

In [11]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration
# KeyBERT
keybert_model = KeyBERTInspired()


# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)


# System prompt describes information given to all conversations
system_prompt = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for labeling topics.
<</SYS>>
"""
main_prompt = """
[INST]
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.
[/INST]
"""

example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.

[/INST] Environmental impacts of eating meat
"""

prompt = system_prompt+ example_prompt + main_prompt


llama2 = TextGeneration(generator, prompt=prompt)
representation_model = {
    "Llama2": llama2,
    "MMR": mmr_model
}

In [12]:
from sentence_transformers import SentenceTransformer
df['processed_comments_review'] = df['processed_comments_review'].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en")
embeddings_30k = embedding_model.encode(df['processed_comments_review'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/106 [00:00<?, ?it/s]

In [18]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model_30k = UMAP(n_neighbors=5, n_components=4, min_dist=0., metric='cosine', random_state=42)
hdbscan_model_30k = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [19]:
from bertopic import BERTopic
import nltk
from bertopic.vectorizers import ClassTfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 

ctfidf_model = ClassTfidfTransformer()
vectorizer_model = CountVectorizer(stop_words="english")

topic_model_30k = BERTopic(

  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model_30k,
  hdbscan_model=hdbscan_model_30k,
  vectorizer_model=vectorizer_model,
  ctfidf_model=ctfidf_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics_30k, probs_30k = topic_model_30k.fit_transform(df['processed_comments_review'].tolist(), embeddings_30k)
print(topics_30k[:10])

2025-02-02 16:13:11,910 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-02 16:13:18,803 - BERTopic - Dimensionality - Completed ✓
2025-02-02 16:13:18,804 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-02 16:13:18,875 - BERTopic - Cluster - Completed ✓
2025-02-02 16:13:18,876 - BERTopic - Representation - Extracting topics from clusters using representation models.
 18%|█▊        | 10/56 [05:17<24:18, 31.71s/it]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [15]:
# Save the DataFrame to a CSV file
if False:
    df = pd.DataFrame(topic_model_30k.get_topic_info())
    df.to_csv('bertopic_output_30k.csv', index=False)
    
# Show topics
topic_model_30k.get_topic_info()

,Topic,Count,Name,Representation,Llama2,MMR,Representative_Docs
0,-1,17,-1_section_wanted_appreciation_express,"[section, wanted, appreciation, express, seria...","[Serialization of configuration sections, , , ...","[section, wanted, appreciation, express, seria...","[\n, , The other day I tried to serialize a vo..."
1,0,3211,0_use_think_don_just,"[use, think, don, just, need, device, entity, ...","[\nEnvironmental impacts of eating, but with t...","[need, device, change, code, config, integrati...",[Let's define this as a `namedtuple` to make i...
2,1,95,1_sneak_tessie_teslemetry_walrus,"[sneak, tessie, teslemetry, walrus, forget, sp...","[Sneaky Walrus Reverts, , , , , , , , , ]","[sneak, tessie, teslemetry, walrus, speed, rev...","[, \r\nDon't forget the walrus!, What entity d..."
3,2,66,2_decision_configuration_integration_yaml,"[decision, configuration, integration, yaml, a...","[Integration Refactoring Required, , , , , , ,...","[decision, yaml, adr, flow, config, pr, change...",[We no longer allow integrations to add or cha...


In [16]:
df["topic"] = topics_30k
df.iloc[0:20]

,PR Number,processed_comments_review,processed_comments_issue,topic
0,129755,Tests are missing. Waiting on an intents bump ...,[Why is this added to the November release mil...,0
1,129675,"This are glob pattern not regexes, so the {6} ...","[Do we need a docs PR here?, > Do we need a do...",0
2,129299,Please reduce to one platform #home-assistant-...,[I made the single PR for the whole lg_thinq i...,0
3,129232,I think a few of these are bugfixes that shoul...,[There are quite a few tests which does a reau...,0
4,129088,This can now be moved to a constant at the top...,[@epenet What's the advantage of `suggested_va...,0
5,128984,Can we let this function return a list instead...,[I'll add more tests once I get some feedback ...,0
6,128947,,[drafting for one more lib bump since the CO2 ...,1
7,128942,We normally only want to import from the integ...,"[Your force push overwrote my commit., > Your ...",0
8,128919,"let's make the domain `music_assistant` Hah, I...",[Small note: the coverage for config_flow.py d...,0
9,128909,Are both of these exceptions realistic? When s...,"[> Also, I am now wondering, does `nest` suppo...",0
